In [13]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch
from langchain import PromptTemplate, LLMChain
from langchain.llms import HuggingFacePipeline
from langchain.docstore.document import Document
from langchain.chains.summarize import load_summarize_chain
from langchain.chains.mapreduce import MapReduceChain
from langchain.text_splitter import TokenTextSplitter

import spacy

from math import ceil
from spacy.lang.en import English 

In [ ]:
from spacy.lang.en import English 

nlp = English()
nlp.add_pipe('sentencizer')


def split_in_sentences(text):
    doc = nlp(text)
    return [str(sent).strip() for sent in doc.sents]


split_in_sentences(doc)

In [11]:
from langchain.text_splitter import TokenTextSplitter, CharacterTextSplitter

text_splitter = CharacterTextSplitter()

with open("/home/ning/Data/Dropbox/Working_Directory/NLP/langchain_llamacpp/data/feature/test1_0.txt") as f:
    doc = f.read()

texts = text_splitter.split_text(doc)

docs = [Document(page_content=t) for t in texts[:3]]

# chain = load_summarize_chain(llm, chain_type="map_reduce", return_intermediate_steps=True)
# chain = load_summarize_chain(llm, chain_type="stuff")

# result = chain(docs)
# print(i)
# print(result)

In [12]:
len(texts)

2

In [ ]:
result['intermediate_steps']

In [ ]:
%reset -f

from langchain.prompts import PromptTemplate
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)

In [ ]:
prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)

prompt

In [ ]:
prompt = PromptTemplate(
    input_variables=["company", "product"],
    template="What is a good name for {company} that makes {product}?",
)

prompt

In [ ]:
human_message_prompt = HumanMessagePromptTemplate(
        prompt=PromptTemplate(
            template="What is a good name for a company that makes {product}?",
            input_variables=["product"],
        )
    )

chat_prompt_template = ChatPromptTemplate.from_messages([human_message_prompt])

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_name_or_path = "TheBloke/Mistral-7B-v0.1-GPTQ"
# To use a different branch, change revision
# For example: revision="gptq-4bit-32g-actorder_True"
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision="main")

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

prompt = "Tell me about AI"
prompt_template=f'''{prompt}
'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
print(tokenizer.decode(output[0]))

# Inference can also be done using transformers' pipeline

print("*** Pipeline:")
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    top_k=40,
    repetition_penalty=1.1
)

print(pipe(prompt_template)[0]['generated_text'])
